# Rabi measurement and pulsed ODMR

In [35]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

In [36]:
try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizerlogic
except NameError:
    manager.startModule('logic', 'optimizerlogic')
try:
    sequencegeneratorlogic
except NameError:
    manager.startModule('logic', 'sequencegeneratorlogic')
try:
    pulsedmeasurement
except NameError:
    manager.startModule('gui', 'pulsedmeasurement')    

In [37]:
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [42]:
# static hardware parameters:
setup = OrderedDict()
setup['sampling_freq'] = pulsedmeasurementlogic.sample_rate
setup['fc_binwidth'] = pulsedmeasurementlogic.fast_counter_binwidth
setup['wait_length'] = 1.5e-6
setup['aom_delay'] = 500e-9
#setup['channel_config_name'] = pulsedmeasurement.current_channel_config_name
setup['seq_trig']=''
setup['gate']='d_ch2'
setup['NV_name']='NV1'
setup['poi']='poi_20170908_1920_43_092302'
setup['laser_length'] = 3.0e-6
setup["min_counts"] = 40000 # if after a refocus only 40kcounts are measured the refocus is repeated up to max 3 times
# Set amplitude in logic and pulser hardware:
amp_dict = OrderedDict()
amp_dict['a_ch1'] = 0.25
amp_dict['a_ch2'] = 0.0
AWG5014C.amplitude_dict = amp_dict
#pulsedmaster.pulse_generator_settings_changed(setup['sampling_freq'], setup['channel_config_name'], amp_dict, False)

In [10]:
# for pulsed ODMR:
odmr_dict = OrderedDict()
odmr_dict['pi_length'] = 200.3e-9
odmr_dict['freq_res'] = 2e+5
odmr_dict['points'] = 100
odmr_dict['measurement_time'] = 20
odmr_dict['fit_function']='Pulsed ODMR'

mw_freq = 1398.02e+6 # in MHz
mw_power = -14

freq =  (mw_freq-0.1e+9)-odmr_dict['points']*odmr_dict['freq_res']/2

freq = 1.39e+9

pulsedmeasurement._pa.ext_control_mw_freq_DoubleSpinBox.setValue(freq)
pulsedmeasurement._pa.ext_control_mw_power_DoubleSpinBox.setValue(mw_power)
pulsedmeasurement.ext_mw_params_changed()

In [54]:
# for Rabi:
rabi = OrderedDict()
rabi['mw_amp'] = 0.25
rabi['mw_freq'] = 100.0e6 
rabi['tau_start'] = 300e-9
rabi['tau_step'] = 10e-9
rabi['number_of_taus'] = 100
rabi['measurement_time'] = 30


In [55]:
mw_freq = 1243.62e+6 # in MHz
mw_power = -13

pulsedmeasurement._pa.ext_control_mw_freq_DoubleSpinBox.setValue(mw_freq-0.1e+9)
pulsedmeasurement._pa.ext_control_mw_power_DoubleSpinBox.setValue(mw_power)
pulsedmeasurement.ext_mw_params_changed()

In [57]:
do_rabi()

(False, 1.3599353381002657e-07, 0.27541912625755555, 7353290.7924646605, 0.83442109424175959)

In [48]:
pulsedmasterlogic.save_measurement_data('s','Rabi_'+setup['NV_name']+'_'+str(rabi['mw_amp'])+'V',True)

In [8]:
def do_rabi(generate_new=True, save_tag=None):
    
    length = rabi['number_of_taus'] * rabi['tau_step']
            
    # sanity check for long sequences:
    if length > 3e-3:
        print("Rabi exceeds 3 ms too long for AWG5014C")
        return True,0,0,0,0
    # generate ensemble object
#     rabi['tau_res'] = rabi['length'] / (rabi['points'] - 1)
    tau_arr = np.arange(rabi['number_of_taus']) * rabi['tau_step']+rabi['tau_start']
    if generate_new:
        sequencegeneratorlogic.delete_ensemble('Rabi')
        sequencegeneratorlogic.generate_rabi('Rabi', rabi['tau_start'], rabi['tau_step'], rabi['number_of_taus'], rabi['mw_freq'], 
                                        rabi['mw_amp'], 'a_ch1',setup['laser_length'] , 1.0, setup['aom_delay'], 
                                        setup['wait_length'], setup['seq_trig'], setup['gate'])
        if 'Rabi' not in sequencegeneratorlogic.saved_pulse_block_ensembles:
            sequencegeneratorlogic.log.error('Ensemble generation of Rabi in notebook timed out.')
            return
        pulsedmasterlogic.sample_block_ensemble('Rabi', True)
    else:
        pulsedmasterlogic.load_asset_into_channels('Rabi')
    while pulsedmasterlogic.status_dict['sauplo_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(0.2)
    pulsedmasterlogic.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, rabi['number_of_taus'], 100e-6, [], False)
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length']+0.2e-6)
    pulsedmasterlogic.analysis_interval_changed(1)
    # perform measurement
    pulsedmasterlogic.start_measurement()
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.2)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < rabi['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmasterlogic.manually_pull_data() 
    time.sleep(1)
    pulsedmasterlogic.stop_measurement()
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.2)
        time.sleep(2)
    # do fit
    x, y, param_dict = pulsedmeasurementlogic.do_fit('Rabi')
    new_freq = param_dict.params['frequency'].value
    contrast = 2.*param_dict.params['amplitude'].value
    offset = param_dict.params['offset'].value
    period = 1./new_freq
    
#     # save data
#     if save_tag is None:
#         pulsedmasterlogic.save_measurement_data('s','Rabi_'+setup['NV_name']+'_'+str(rabi['mw_amp'])+'V',True)
#     else:
#         pulsedmasterlogic.save_measurement_data('s',save_tag,True)
    
    # write logfile
    write_to_logfile('Rabi_log', str(datetime.datetime.now()),name= setup['NV_name'], amplitude=rabi['mw_amp'], period=period,
                     frequency=new_freq, contrast=contrast, offset= offset)
    return user_terminated, period, contrast, new_freq, offset

In [32]:
def do_odmr(generate_new=True,save_data=False):
    # generate ensemble object
    if odmr_dict['pi_length']>3e-3:
        print("Pulsed ODMR exceeds length of 3ms")
        return
        
    freq_arr = (mw_freq-odmr_dict['points']*odmr_dict['freq_res']/2) + np.arange(odmr_dict['points']) * odmr_dict['freq_res']    
        
    if generate_new:
        sequencegeneratorlogic.delete_ensemble('PulsedODMR')
        sequencegeneratorlogic.generate_pulsed_odmr('PulsedODMR', odmr_dict['pi_length'], 100.0e6, odmr_dict['freq_res'], 
                                              odmr_dict['points'], 0.25, 'a_ch1', setup['laser_length'] , 
                                              1.0, setup['aom_delay'], setup['wait_length'], setup['seq_trig'], setup['gate'])
        if 'PulsedODMR' not in sequencegeneratorlogic.saved_pulse_block_ensembles:
            sequencegeneratorlogic.log.error('Ensemble generation of PulsedODMR from notebook did not work.')
            return
        pulsedmasterlogic.sample_block_ensemble('PulsedODMR', True)
    else:
        pulsedmasterlogic.load_asset_into_channels('PulsedODMR')
    while pulsedmasterlogic.status_dict['sauplo_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(0.5)
    # delete old fit
    pulsedmasterlogic.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmasterlogic.measurement_sequence_settings_changed(freq_arr, odmr_dict['points'], 100e-6, [], False)
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length']+0.2e-6)
    pulsedmasterlogic.analysis_interval_changed(1)
    # perform measurement
    pulsedmasterlogic.start_measurement()
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < odmr_dict['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(1)
    pulsedmasterlogic.manually_pull_data() 
    time.sleep(1)
    pulsedmasterlogic.stop_measurement()
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
#    pulsedmaster.save_measurement_data('Hz','PulsedODMR'+setup['NV_name'])
    time.sleep(2)
    if odmr_dict['fit_function']=='Pulsed ODMR':
        x, y, param_dict= pulsedmeasurementlogic.do_fit('Pulsed ODMR')
        new_freq = param_dict.params['center'].value
        contrast = param_dict.params['contrast'].value
        linewidth = param_dict.params['fwhm'].value
            
    if save_data:
        pulsedmasterlogic.save_measurement_data('Hz','ODMR_'+setup['NV_name'],True)
    # write logfile
    write_to_logfile('PulsedODMR_log', str(datetime.datetime.now()),name= setup['NV_name'], frequency=new_freq,
                     contrast=contrast,linewidth=linewidth)
    return user_terminated, new_freq, contrast, linewidth

In [33]:
do_odmr()

(False, 1395623678.5516646, -4.6706459250703105, 1391792.1485044595)